# Create parameters and mechanism json files

This notebook creates the `mechanisms.json`, `parameters.json`, and `parameters_release.json` from the `pyr_mainaxon_reboot.json` file provided by Dr. Zbili.

In [ ]:
import json
from pprint import pprint
from copy import copy
import numpy as np

In [ ]:
param_file = "parameters.json"
mechanism_file = "mechanisms.json"
param_release_file = "parameters_release.json"
param_info_file = "pyr_mainaxon_reboot.json"

In [ ]:
save_files = True

In [ ]:
param_info = json.load(open(param_info_file))
pprint(param_info)

In [ ]:
sections_mech_map = {
    "basal": ["basal", "somadend"], # all dendrites are "basal" and active
    "somatic": ["somatic", "somadend"],
    "axonal": ["axonal"],
    "axon_initial_segment": ["axon_initial_segment"]
    }

## Create mechanisms.json

In [ ]:
mechanisms = {}
for sec, param_secs in sections_mech_map.items():
    mechanisms[sec] = []
    for param_sec in param_secs:
        mechanisms[sec].extend(param_info["mechanisms"][param_sec]["mech"])
mechanisms["all"] = ["pas"]
pprint(mechanisms)

if save_files:
    json.dump(mechanisms, open(mechanism_file, "w"), indent=4)

## Create parameters.json and parameters_release.json

For the `parameters_release.json`, the mean of the `bounds` is used.

In [ ]:
params = []
params_release = []

# set global params
for par in param_info["parameters"]["global"]:
    par_dict = {"param_name": par["name"],
                "type": "global",
                "value": par["val"]}
    params.append(par_dict)
    params_release.append(par_dict) 
    
# add "all" specific parameters
mech_all = param_info["mechanisms"]["all"]
for par_all in param_info["parameters"]["all"]:
    par_dict = {"param_name": par_all["name"],
                "type": "section",
                "sectionlist": "all",
                "dist_type": "uniform"}
    par_dict_release = copy(par_dict)
    # check if a mech needs to be added
    for mech in mech_all["mech"]:
        if mech in par_dict["param_name"]:
            par_dict["mech"] = mech
    # add values
    if isinstance(par_all["val"], list):
        par_dict["bounds"] = par_all["val"]
        par_dict_release["value"] = np.mean(par_all["val"])
    else: # scalar
        par_dict["value"] = par_all["val"]
        par_dict_release["value"] = par_all["val"]
    params.append(par_dict)
    params_release.append(par_dict_release)    

    
# add section specific mechanisms
for sec, param_secs in sections_mech_map.items():
    for param_sec in param_secs:
        print("\nSectionlist", param_sec)
        mechs = param_info["mechanisms"][param_sec]
        for par in param_info["parameters"][param_sec]:
            par_dict = {"param_name": par["name"],
                        "type": "section",
                        "sectionlist": sec
                       }
            if "dist" not in par:
                par_dict.update({"dist_type": "uniform"})
            else:
                par_dict.update({"dist_type": par["dist"]})
                par_dict.update({"dist": param_info["distributions"][par["dist"]]["fun"]})
                for dist_par in param_info["distributions"][par["dist"]]:
                    if dist_par != "__comment" and dist_par != "fun":
                        par_dict.update({dist_par: param_info["distributions"][par["dist"]][dist_par]})
                    if dist_par == "parameters":
                        dependencies = []
                        for dep in param_info["distributions"][par["dist"]][dist_par]:
                            dependencies.append(f"{dep}_{par['dist']}")
                        par_dict["dependencies"] = dependencies
                par_dict["type"] = "range"
            par_dict_release = copy(par_dict)
            # check if a mech needs to be added
            for mech in mechs["mech"]:
                if mech in par_dict["param_name"]:
                    par_dict["mech"] = mech
                    mech_param = par_dict["param_name"][:par_dict["param_name"].find(("_" + mech))]
                    par_dict["mech_param"] = mech_param
                    
            # add values
            if isinstance(par["val"], list):
                par_dict["bounds"] = par_all["val"]
                par_dict_release["value"] = np.mean(par_all["val"])
            else: # scalar
                par_dict["value"] = par["val"]
                par_dict_release["value"] = par["val"]
                            
            params.append(par_dict)
            params_release.append(par_dict_release) 

# add distribution param
distributions = param_info["distributions"]
for sectionlist in param_info["parameters"]:
    if "distribution" in sectionlist:
        for par in param_info["parameters"][sectionlist]:
            dist_name = sectionlist.split('distribution_')[1]
            par_dict = {"param_name": f"{par['name']}_{dist_name}",
                        "type": "meta",
                       }
            dist = distributions[dist_name]
            par_dict.update({"dist_type": dist_name})
            par_dict.update({"dist": param_info["distributions"][dist_name]["fun"]})
            for dist_par in param_info["distributions"][dist_name]:
                if dist_par != "__comment" and dist_par != "fun":
                    par_dict.update({dist_par: param_info["distributions"][dist_name][dist_par]})
            par_dict_release = copy(par_dict)
            
            # add values
            if isinstance(par["val"], list):
                par_dict["bounds"] = par_all["val"]
                par_dict_release["value"] = np.mean(par_all["val"])
            else: # scalar
                par_dict["value"] = par["val"]
                par_dict_release["value"] = par["val"]
            
            params.append(par_dict)
            params_release.append(par_dict_release)


if save_files:         
    json.dump(params, open(param_file, "w"), indent=4)
    json.dump(params_release, open(param_release_file, "w"), indent=4)

In [ ]:
dist

In [ ]:
param_info["parameters"]["distribution_decay"]

In [ ]:
debug

In [ ]:
pprint(params)

In [ ]:
pprint(params_release)

In [ ]:
# function from Mickael
def define_parameters(params_filename, stage=None, past_params=[]):
    """Define parameters"""
    parameters = []
    # Fixed section parameters
    # TODO check the order of executions of all parameters
    with open(os.path.join(os.path.dirname(__file__), '..', params_filename)) as params_file:
        definitions = json.load(
            params_file,
            object_pairs_hook=collections.OrderedDict)
    # set distributions
    distributions = collections.OrderedDict()
    distributions["uniform"] = ephys.parameterscalers.NrnSegmentLinearScaler()
    distributions_definitions = definitions["distributions"]
    for distribution, definition in distributions_definitions.iteritems():
        if "parameters" in definition:
            dist_param_names = definition["parameters"]
        else:
            dist_param_names = None
        distributions[distribution] = \
            parameterscaler_custom.NrnSegmentSomaDistanceScaler(
                name=distribution ,distribution=definition["fun"],
                dist_param_names=dist_param_names,soma_ref_point=definition["soma_ref_point"])
    params_definitions = definitions["parameters"]
    if "__comment" in params_definitions:
        del params_definitions["__comment"]
    for sectionlist, params in params_definitions.iteritems():
        if sectionlist == 'global':
            seclist_locs = None
            is_global = True
            is_dist = False
        elif 'distribution_' in sectionlist:
            is_dist = True
            seclist_locs = None
            is_global = False
            dist_name = sectionlist.split('distribution_')[1]
            dist = distributions[dist_name]
        else:
            seclist_locs = multi_locations(sectionlist)
            is_global = False
            is_dist = False
        bounds = None
        value = None
        for param_config in params:
            param_name = param_config["name"]
            if isinstance(param_config["val"], (list, tuple)):
                full_name = '%s.%s' % (param_name, sectionlist)
                # check and define stage of this parameter,
                # if not given, stage is 1
                if "stage" in param_config:
                    this_stage = param_config["stage"]
                else:
                    this_stage = [1]
                if (stage is None):
                    # used for analysis, use previous values or leave as bounds
                    if full_name in past_params:
                        is_frozen = True
                        value = past_params[full_name]
                        bounds = None
                        logger.debug(
                            'Param %s, use value %f from past_params' % (full_name, value))
                    else: # optimize
                        is_frozen = False
                        bounds = param_config["val"]
                        value = None
                elif (stage in this_stage):
                    # use for optimization here
                    is_frozen = False
                    bounds = param_config["val"]
                    value = None
                elif stage > max(this_stage):
                    # optimization was done in an earlier stage
                    # use previous values
                    is_frozen = True
                    value = past_params[full_name]
                    bounds = None
                    logger.debug(
                        'Param %s, use value %f from stage %s, stage now %s' % (full_name, value, this_stage, stage))
                elif stage < min(this_stage):
                    # not yet fitted yet, set to 0
                    is_frozen = True
                    value = 0
                    bounds = None
                    logger.debug(
                        'Param %s, not yet used, set to 0, use in %s, stage now %s' % (full_name, this_stage, stage))
            else:
                is_frozen = True
                value = param_config["val"]
                bounds = None
            if is_global:
                parameters.append(
                    ephys.parameters.NrnGlobalParameter(
                        name=param_name,
                        param_name=param_name,
                        frozen=is_frozen,
                        bounds=bounds,
                        value=value))
            elif is_dist:
                parameters.append(
                    ephys.parameters.MetaParameter(
                        name='%s.%s' % (param_name, sectionlist),
                        obj=dist,
                        attr_name=param_name,
                        frozen=is_frozen,
                        bounds=bounds,
                        value=value))
            else:
                if "dist" in param_config:
                    dist = distributions[param_config["dist"]]
                    use_range = True
                else:
                    dist = distributions["uniform"]
                    use_range = False
                if use_range:
                    parameters.append(ephys.parameters.NrnRangeParameter(
                        name='%s.%s' % (param_name, sectionlist),
                        param_name=param_name,
                        value_scaler=dist,
                        value=value,
                        bounds=bounds,
                        frozen=is_frozen,
                        locations=seclist_locs))
                else:
                    parameters.append(ephys.parameters.NrnSectionParameter(
                        name='%s.%s' % (param_name, sectionlist),
                        param_name=param_name,
                        value_scaler=dist,
                        value=value,
                        bounds=bounds,
                        frozen=is_frozen,
                        locations=seclist_locs))
    return parameters